In [ ]:
# packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from IPython.display import HTML, display
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

train = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/train.csv")


## Introduction

1. EDA + data visualization
2. Python Image (IPython.display, PIL )
3. * CNN, neutral networks， detectron, yolo, mask rcnn

In [ ]:
from IPython.display import Image
from IPython.display import HTML, display
from PIL import Image, ImageFilter
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
data_dr="../input/petfinder-pawpularity-score"
train=pd.read_csv(data_dr+"/train.csv")
train.head()

In [ ]:
train.describe()

In [ ]:
# Cut Level for Powpularity
powcat=pd.cut(train.Pawpularity, bins=[0,40,60,85,100], 
             labels=["Open_B16", "Sweet" ,"Cute", "Home"])
train.insert(14,"Powcat", powcat)

In [ ]:
cuteness=pd.cut(train.Pawpularity, bins=[0,40,60,85,100], 
             labels=[0, 1, 2, 3])
train.insert(15,"Cuteness", cuteness)


In [ ]:
train.head()

In [ ]:
sns.set()
sns.countplot(train["Powcat"],palette="Set2")

In [ ]:
fig=plt.figure()
ax=sns.violinplot(x="Powcat" , y="Pawpularity",palette="Set2", data=train)
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(15,5)})
fig=plt.figure()
sns.histplot(data=train, x="Pawpularity",kde=True, bins=100)
plt.axvline(train["Pawpularity"].mean(),c="pink",ls="-", lw=3, label="Mean")
plt.axvline(train["Pawpularity"].median(),c="darkseagreen",ls="-",lw=3,label="Median")
plt.title("Distribution of Pawpularity Scores", fontsize=20,fontweight="bold")
plt.legend()
plt.show()

In [ ]:
pet_ft=train.drop(["Id","Pawpularity","Powcat"], axis=1)
y=train.Pawpularity
plt.figure(figsize=(20,10))
sns.heatmap(train.corr(),annot=True)
plt.show()

In [ ]:
train.head()

In [ ]:
plt.figure(figsize=(10,6))
train[train["Powcat"]=="Open_B16"]['Subject Focus'].hist(alpha=0.5,color='blue',
                                              bins=30,label='Open_B16')
# df[df['converted']==0]['total_pages_visited'].hist(alpha=0.5,color='red',
#                                               bins=30,label='converted=0')
plt.legend()
# plt.xlabel('total_pages_visited')


# "Open_B16", "Sweet" ,"Cute", "Home"

In [ ]:
def fplot(var,label):
    fig, ax=plt.subplots(1,3)
    sns.violinplot(x=var, y="Pawpularity",data=train,palette="Set2",ax=ax[0])
    sns.histplot(train,x="Pawpularity", hue=var, palette="Set2",kde=True,ax=ax[1])
    sns.countplot(x=var,hue="Powcat",data=train,palette="Set2", ax=ax[2])
    plt.suptitle(label)
    return plt
fig.show()  

In [ ]:
fplot("Eyes", "Eye")

In [ ]:
fplot("Face", "Picture with Face")

In [ ]:
fplot("Near", "Pictures with Close position")

In [ ]:
fplot("Action", "Pets Picture with action")

In [ ]:
fplot("Accessory", "Pet's Picture with Accessory")

In [ ]:
fplot("Group", "Pet's Picture with Group")

In [ ]:
fplot("Collage", "Pet's Picture with Collage")

In [ ]:
fplot("Human", "Pet's Picture with Human")

In [ ]:
fplot("Occlusion", "Pet's Picture with Accessory")

In [ ]:
fplot("Blur", "Pet's Picture Blur")

In [ ]:
pca=PCA(n_components=5)
pca.fit(pet_ft)
print(pca.explained_variance_ratio_)
x_pca=pca.fit_transform(pet_ft)
ps=pd.DataFrame(x_pca)
ps.head()

In [ ]:
#try
topc=pd.DataFrame(ps[[3,4]])
print(topc.shape)
print(topc.head())
fig=plt.figure(figsize=(8,8))
plt.plot(topc[3],topc[4],"o")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()

In [ ]:
cloud=KMeans(n_clusters=4,random_state=8).fit(topc)
centers=cloud.cluster_centers_
c_pred=cloud.predict(topc)


In [ ]:
fig=plt.figure(figsize=(8,8))
colors=["orange","purple","blue", "green"]
colored=[colors[k] for k in c_pred]
plt.scatter(topc[3],topc[4], color=colored)
for ci,c in enumerate(centers):
    plt.plot(c[0],c[1],"o",color="red", alpha=0.9)
plt.xlabel("x")
plt.ylabel("y")

plt.show()

In [ ]:
pet_ft=pet_ft.copy()
pet_ft["cluster"]=c_pred
pet_ft["label"]=train.Powcat
pet_ft.head(10)
pd.crosstab(index=pet_ft["cluster"],columns=pet_ft["label"])

**CNN**

In [ ]:
import os
import cv2 as cv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Conv1D, MaxPooling1D
from keras.layers import Dense, Activation, Dropout, Flatten
from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler

**Load the pictures**

In [ ]:
path = '../input/petfinder-pawpularity-score/train/'
training_img = os.listdir(path) # list all training images names
print('There are {} images in the training directory'.format(len(training_img)))
# data_list = pickle.load(open(training_img, "rb"))
print(len(training_img))

In [ ]:
print(training_img[:5])

In [ ]:
model = Sequential()
num_classes = 2

# # TODO: Conv1
model.add(Conv2D(filters=32, kernel_size=7,padding="same",activation="relu",input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=7, strides=4, padding="valid"))
# # TODO: Conv2
model.add(Conv2D(filters=16, kernel_size=5,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=7, strides=4, padding="valid"))
# # TODO: Flatten the layer
model.add(Flatten())
# # TODO: Add the intermediate fully connected layers (Dense in keras)
model.add(Dense(64, activation='relu'))
# # TODO: Add the final fully connected layer with the softmax activation function
model.add(Dense(4, activation='softmax'))

In [ ]:
model.summary()